In [1]:
import pandas as pd

# Example dataframe with 30 rows
data = {'column1': range(1, 31)}
df = pd.DataFrame(data)

# Create a new index for every 5 rows
df['new_index'] = ((df.index) % 5) + 1

# Assign new_index as the dataframe index (if desired)
df.set_index('new_index', inplace=True)

C:\Users\msubki\AppData\Local\Temp\ipykernel_17608\766470758.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df

,column1
new_index,
1,1
2,2
3,3
4,4
5,5
1,6
2,7
3,8
4,9
